In [1]:
import pandas as pd
import importlib
import gc
import sys
sys.path.append('./scripts')  
import dataset_pcp
importlib.reload(dataset_pcp)

<module 'dataset_pcp' from 'c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2025\\Lab3\\Lab3-MCD\\notebooks\\entregable\\./scripts\\dataset_pcp.py'>

In [2]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=',')
df.shape

(2945818, 13)

In [3]:
#### COMBINATORIA ####
data = dataset_pcp.combinatoria_periodo_cliente_producto()
data['periodo'] = data['periodo'].dt.year * 100 + data['periodo'].dt.month
data.shape
# (44388, 2)


(26499636, 3)

In [4]:
#### PRODUCTOS SIN HISTORIA ####
#### Vemos que hay  42 + 7 + 20 productos que tienen historia <= 3 meses. Los elimino.
prod_sin_historia = dataset_pcp.getProductos_sinHistoria()
data = data[~data['product_id'].isin(prod_sin_historia)]
data.shape

(25016688, 3)

In [5]:
#### PRODUCTOS QUE NO NACIERON ####
data = dataset_pcp.eliminarProductos_sinNacer(df, data)
data.shape

(20862165, 4)

In [6]:
#### PRODUCTOS QUE MURIERON ####
data = dataset_pcp.eliminarProductosMuertos(df, data)
data.shape

(18723114, 5)

In [7]:
#### ELIMINAR CLIENTES QUE NO NACIERON
data = dataset_pcp.eliminarClientes_sinNacer(df, data)
data.shape

(17082581, 6)

In [14]:
#### ELIMINAR CLIENTES QUE MURIERON
data = dataset_pcp.eliminarClientesMuertos(df, data)
data.shape

(15487665, 7)

In [8]:
#### 3MESES PRODUCTOS NUEVOS: HISTORIA MENOR A 12 MESES ####
data = dataset_pcp.marcarProductosNuevos_3M(data)
data.shape

(17082581, 10)

In [10]:
data

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial
0,20001,201701,2017-01-01,2019-12-01,1,36,0,0
1,20001,201702,2017-01-01,2019-12-01,2,36,0,0
2,20001,201703,2017-01-01,2019-12-01,3,36,0,0
3,20001,201704,2017-01-01,2019-12-01,4,36,0,0
4,20001,201705,2017-01-01,2019-12-01,5,36,0,0
...,...,...,...,...,...,...,...,...
31357,21281,201704,2017-02-01,2017-08-01,3,7,1,1
31358,21281,201705,2017-02-01,2017-08-01,4,7,1,0
31359,21281,201706,2017-02-01,2017-08-01,5,7,1,0
31360,21281,201707,2017-02-01,2017-08-01,6,7,1,0


In [9]:
#### MERGE CON PRODUCTOS ####
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
data = data.merge(productos, how='left', on="product_id")
del productos

#### MERGE CON STOCKS ####
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
data = data.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks

#### MERGE CON SELLIN ####
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
sellin = sellin.groupby(by=["periodo","product_id", "customer_id"]).agg({"tn":"sum", "plan_precios_cuidados":"sum", "cust_request_qty":"sum", "cust_request_tn":"sum"}).reset_index()
data = data.merge(sellin, how='left', on=['periodo', 'product_id', 'customer_id'])
del sellin
gc.collect()

0

In [17]:
data

,product_id,customer_id,periodo,nacimiento_producto,muerte_producto,nacimiento_cliente,muerte_cliente,mes_n,total_meses,producto_nuevo,...,cat1,cat2,cat3,brand,sku_size,stock_final,tn,plan_precios_cuidados,cust_request_qty,cust_request_tn
0,20001,10234,201701,2017-01-01,2019-12-01,2017-01-01,2019-12-01,1,17776,0,...,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,0.33579,0.0,1.0,0.33579
1,20001,10032,201701,2017-01-01,2019-12-01,2017-01-01,2019-12-01,2,17776,0,...,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,12.31230,0.0,2.0,12.31230
2,20001,10217,201701,2017-01-01,2019-12-01,2017-01-01,2019-12-01,3,17776,0,...,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,NaN,NaN,NaN,NaN
3,20001,10125,201701,2017-01-01,2019-12-01,2017-01-01,2019-11-01,4,17776,0,...,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,0.08954,0.0,1.0,0.08954
4,20001,10012,201701,2017-01-01,2019-12-01,2017-01-01,2019-12-01,5,17776,0,...,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,6.97324,0.0,5.0,6.97324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15487660,21281,10212,201708,2017-02-01,2017-08-01,2017-08-01,2019-12-01,3500,3504,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15487661,21281,10376,201708,2017-02-01,2017-08-01,2017-08-01,2019-12-01,3501,3504,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15487662,21281,10576,201708,2017-02-01,2017-08-01,2017-08-01,2019-11-01,3502,3504,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15487663,21281,10207,201708,2017-02-01,2017-08-01,2017-08-01,2019-12-01,3503,3504,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#### COMPLETO TN CON CEROS ####
####  ¿cuantos?
print(f"Total de periodos con Nan debido a la combinatoria periodo_x_producto: {data['tn'].isna().sum()}")
#### Lo completo con ceros
data['tn'] = data['tn'].fillna(0)

Total de periodos con Nan debido a la combinatoria periodo_x_producto: 14142135


In [ ]:
#### GUARDAR DATAFRAME ####
data["nacimiento_producto"] = data["nacimiento_producto"].dt.year * 100 + data["nacimiento_producto"].dt.month
data["muerte_producto"] = data["muerte_producto"].dt.year * 100 + data["muerte_producto"].dt.month
data["nacimiento_cliente"] = data["nacimiento_cliente"].dt.year * 100 + data["nacimiento_cliente"].dt.month
# data["muerte_cliente"] = data["muerte_cliente"].dt.year * 100 + data["muerte_cliente"].dt.month
data[['nacimiento_producto','muerte_producto']]

In [13]:
data.to_csv("./datasets/periodo_x_cliente_x_producto.csv", index=False, sep=',', encoding='utf-8')

In [14]:
data.sort_values(by=['tn'], ascending=False).head(10)

,product_id,customer_id,periodo,nacimiento_producto,muerte_producto,nacimiento_cliente,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,cat1,cat2,cat3,brand,sku_size,stock_final,tn,plan_precios_cuidados,cust_request_qty,cust_request_tn
37981,20002,10001,201911,201701,201912,201701,18427,19555,0,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,38.41355,547.87849,0.0,36.0,551.56137
11003,20001,10004,201810,201701,201912,201701,11004,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,33.65111,469.45761,0.0,12.0,510.65893
14949,20001,10001,201905,201701,201912,201701,14950,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,113.54449,439.90647,0.0,31.0,439.90647
4568,20001,10004,201710,201701,201912,201701,4569,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,437.37767,0.0,11.0,437.37767
37391,20002,10001,201910,201701,201912,201701,17837,19555,0,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,25.40071,430.90803,0.0,40.0,444.41192
6150,20001,10004,201801,201701,201912,201701,6151,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,414.05146,0.0,17.0,416.64823
14928,20001,10004,201905,201701,201912,201701,14929,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,113.54449,389.02653,0.0,23.0,389.02653
12692,20001,10001,201901,201701,201912,201701,12693,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,544.71331,386.60688,0.0,53.0,393.26092
9901,20001,10004,201808,201701,201912,201701,9902,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,384.82574,0.0,8.0,384.82574
3006,20001,10004,201707,201701,201912,201701,3007,19555,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,379.44270,0.0,4.0,407.02225
